In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from gensim.models import FastText
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

In [5]:
df = pd.read_csv("data_after_cleansing_normalized.csv")

In [6]:
df.head()

,Tweet,HS,normalized_tweet,clean_tweet
0,- disaat semua cowok berusaha melacak perhatia...,1,- di saat semua pria berusaha melacak perhatia...,di saat semua pria berusaha melacak perhatian...
1,RT USER: USER siapa yang telat ngasih tau elu?...,0,RT USER: USER siapa telat memberi tau elu?eda...,rt user user siapa telat memberi tau eluedan s...
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0,"41. Kadang aku berfikir, kenapa aku tetap perc...",kadang aku berfikir kenapa aku tetap percaya ...
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,user user aku itu akunnku tau matamu sipit tap...
4,USER USER Kaum cebong kapir udah keliatan dong...,1,USER USER Kaum kecebong kafir sudah kelihatan ...,user user kaum kecebong kafir sudah kelihatan ...


In [7]:
X = df['clean_tweet'] #Text
y = df['HS'] #Label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Fast Text
**embedding_size** adalah dimensi dari vektor fitur yang dihasilkan oleh model FastText. Nilai ini menentukan berapa banyak dimensi yang digunakan untuk merepresentasikan setiap kata dalam ruang vektor.

**window_size** adalah parameter yang menentukan jumlah kata yang dipertimbangkan oleh model saat melihat konteks sebuah kata. Misalnya, jika window_size diatur ke 5, itu berarti model akan mempertimbangkan 5 kata sebelum dan sesudah kata yang sedang diproses saat membangun representasi vektor untuk kata tersebut.

**min_word** adalah ambang batas yang menentukan jumlah minimum kemunculan kata dalam korpus teks. Kata-kata yang kurang dari jumlah minimum ini tidak akan dipertimbangkan dalam pembentukan vektor.

**down_sampling** adalah parameter yang mengendalikan proses pengurangan frekuensi kata dalam model untuk mengurangi bobot kata yang sangat sering muncul

In [8]:
# FastText untuk ekspansi fitur
embedding_size = 100
window_size = 5
min_word = 5
down_sampling = 1e-2
fasttext_model = FastText(sentences=X_train, vector_size=embedding_size, window=window_size, min_count=min_word, sample=down_sampling)

In [9]:
# Tokenisasi teks dan lakukan padding
max_len = 100 # Panjang maksimum dari setiap kalimat
X_train_seq = [[fasttext_model.wv[word] for word in sentence.split() if word in fasttext_model.wv] for sentence in X_train]
X_test_seq = [[fasttext_model.wv[word] for word in sentence.split() if word in fasttext_model.wv] for sentence in X_test]
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, dtype='float32')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, dtype='float32')

In [10]:
# Model LSTM
model = Sequential()
model.add(LSTM(128, input_shape=(max_len, embedding_size)))
model.add(Dense(1, activation='sigmoid'))

In [11]:
# Kompilasi model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
model.fit(X_train_pad, y_train, epochs=10, batch_size=32, validation_split=0.1)

Epoch 1/10
414/414 [==============================] - 24s 53ms/step - loss: 0.6736 - accuracy: 0.5960 - val_loss: 0.6741 - val_accuracy: 0.5849
Epoch 2/10
414/414 [==============================] - 23s 55ms/step - loss: 0.6720 - accuracy: 0.5973 - val_loss: 0.6729 - val_accuracy: 0.5856
Epoch 3/10
414/414 [==============================] - 23s 55ms/step - loss: 0.6712 - accuracy: 0.5978 - val_loss: 0.6712 - val_accuracy: 0.5863
Epoch 4/10
414/414 [==============================] - 24s 57ms/step - loss: 0.6707 - accuracy: 0.5981 - val_loss: 0.6724 - val_accuracy: 0.5842
Epoch 5/10
414/414 [==============================] - 23s 57ms/step - loss: 0.6701 - accuracy: 0.5982 - val_loss: 0.6715 - val_accuracy: 0.5849
Epoch 6/10
414/414 [==============================] - 23s 56ms/step - loss: 0.6697 - accuracy: 0.5980 - val_loss: 0.6730 - val_accuracy: 0.5842
Epoch 7/10
414/414 [==============================] - 23s 56ms/step - loss: 0.6692 - accuracy: 0.5982 - val_loss: 0.6714 - val_accuracy:

In [13]:
# Evaluasi model pada data uji
loss, accuracy = model.evaluate(X_test_pad, y_test)
print("Test Accuracy:", accuracy)

115/115 [==============================] - 2s 21ms/step - loss: 0.6745 - accuracy: 0.5891
Test Accuracy: 0.5891304612159729


In [14]:
# Prediksi pada data uji
predictions = model.predict(X_test_pad)

115/115 [==============================] - 3s 21ms/step


In [16]:
# Ambil label prediksi (dibulatkan menjadi 0 atau 1)
y_pred = np.round(predictions).flatten()

In [17]:
from sklearn.metrics import classification_report
print('\nClassification Report\n')
print(classification_report(y_test, y_pred, target_names=['0','1']))


Classification Report

              precision    recall  f1-score   support

           0       0.59      0.99      0.74      2167
           1       0.52      0.01      0.02      1513

    accuracy                           0.59      3680
   macro avg       0.55      0.50      0.38      3680
weighted avg       0.56      0.59      0.44      3680



#### tunning parameter

In [ ]:
=========

In [18]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [19]:
def create_model(units=128, dropout_rate=0.2):
    model = Sequential()
    model.add(LSTM(units, input_shape=(max_len, embedding_size)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [20]:
# Membuat model Keras Classifier untuk digunakan dalam GridSearchCV
model = KerasClassifier(build_fn=create_model)

C:\Users\gilan\AppData\Local\Temp\ipykernel_16392\2534670821.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model)


In [21]:
# Parameter yang akan diuji
parameters = {
    'units': [64, 128, 256],
    'dropout_rate': [0.2, 0.3, 0.4]
}

In [22]:
# Membuat objek GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=parameters, cv=3)

In [23]:
# Melakukan penyetelan parameter pada data latih
grid_search.fit(X_train_pad, y_train)

460/460 [==============================] - 33s 68ms/step - loss: 0.6732 - accuracy: 0.5962


GridSearchCV(cv=3,
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x000002182B5BED90>,
             param_grid={'dropout_rate': [0.2, 0.3, 0.4],
                         'units': [64, 128, 256]})

In [24]:
# Menampilkan hasil terbaik
print("Best Parameters:", grid_search.best_params_)
print("Best Accuracy:", grid_search.best_score_)

Best Parameters: {'dropout_rate': 0.2, 'units': 64}
Best Accuracy: 0.5967652797698975


In [25]:
# Membuat model Keras dengan parameter terbaik
best_units = grid_search.best_params_['units']
best_dropout_rate = grid_search.best_params_['dropout_rate']

best_model = create_model(units=best_units, dropout_rate=best_dropout_rate)

# Melatih model dengan seluruh data latih
best_model.fit(X_train_pad, y_train, epochs=10, batch_size=32)

# Evaluasi model pada data uji
loss, accuracy = best_model.evaluate(X_test_pad, y_test)
print("Test Accuracy with Best Parameters:", accuracy)

Epoch 1/10
460/460 [==============================] - 29s 59ms/step - loss: 0.6734 - accuracy: 0.5953
Epoch 2/10
460/460 [==============================] - 29s 63ms/step - loss: 0.6713 - accuracy: 0.5970
Epoch 3/10
460/460 [==============================] - 28s 61ms/step - loss: 0.6707 - accuracy: 0.5974
Epoch 4/10
460/460 [==============================] - 28s 61ms/step - loss: 0.6701 - accuracy: 0.5976
Epoch 5/10
460/460 [==============================] - 28s 62ms/step - loss: 0.6692 - accuracy: 0.5976
Epoch 6/10
460/460 [==============================] - 28s 62ms/step - loss: 0.6690 - accuracy: 0.5979
Epoch 7/10
460/460 [==============================] - 29s 62ms/step - loss: 0.6688 - accuracy: 0.5975
Epoch 8/10
460/460 [==============================] - 28s 62ms/step - loss: 0.6678 - accuracy: 0.5984
Epoch 9/10
460/460 [==============================] - 28s 62ms/step - loss: 0.6671 - accuracy: 0.5994
Epoch 10/10
115/115 [==============================] - 3s 26ms/step - loss: 0.6733